In [ ]:
import pandas as pd
import numpy as np

In [1119]:
skaters = pd.read_csv(f'cleaned/{2022}{2022+1}_skaters.csv', encoding='utf-8')
skaters = skaters.rename(columns={'Date_x': 'Date', 'Venue_x': 'Venue', 'TOI_x': 'TOI', 'Opposition_x': 'Opposition'})
skaters.to_csv(f'cleaned/{2022}{2022+1}_skaters.csv')


In [1159]:
master = []
for year in range(2007, 2023):
    skaters = pd.read_csv(f'cleaned/{year}{year+1}_skaters.csv', encoding='utf-8')
    skaters['Season'] = str(year) + str(year+1)
    skaters['Player'] = skaters['Player'].str.replace('\xa0', ' ')
    skaters['Date'] = skaters['Date'].str.replace('/', '-')
    skaters.to_csv(f'cleaned/{year}{year+1}_skaters.csv')
    master.append(skaters)
skaters['Season'] = skaters['Season'].astype(object)
skaters = pd.concat(master)
skaters = skaters.reset_index(drop=True)
skaters.to_csv('cleaned/master_skaters.csv', encoding='utf-8')

In [1208]:
skaters = pd.read_csv(f'cleaned/master_skaters.csv')
skaters['exGF'] = skaters['TOI']*0.032+0.055
skaters['exGA'] = skaters['TOI']*0.032+0.055
skaters['eGF'] = skaters['TOI']*0.032+0.055
skaters['eGA'] = skaters['TOI']*0.032+0.055
skaters['GFAA'] = skaters['GF'] - skaters['eGF']
skaters['GAAA'] =  skaters['eGA'] - skaters['GA']
skaters['xGFAA'] = skaters['xGF'] - skaters['exGF']
skaters['xGAAA'] =  skaters['exGA'] - skaters['xGA']

In [1209]:
skaters['Production'] = (0.75*skaters['Goals']) + (0.7*skaters['First Assists']) + (0.55*skaters['Second Assists']) + (0.5*skaters['ixG']) + (0.05*skaters['Shots Blocked']) 
skaters['Misc'] = (0.15*(skaters['Penalties Drawn'] - skaters['Total Penalties'])) + (0.01*(skaters['Faceoffs Won'] - skaters['Faceoffs Lost']))
skaters['Offense'] = np.where(skaters['Position'] == 'D', 
((0.425*(skaters['GF'] + skaters['GFAA'])) + (1.7*(skaters['xGF'] + skaters['xGFAA'])))/2.125 - 0.75,
(((0.625*(skaters['GF'] + skaters['GFAA'])) + (0.625*(skaters['xGF'] + skaters['xGFAA'])))/2) - 0.4)
skaters['Defense'] = np.where(skaters['Position'] == 'D', 
(((0.575*(-skaters['GA'] + skaters['GAAA'])) + (2.3*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 1.05, 
(((0.4375*(-skaters['GA'] + skaters['GAAA'])) + (1.7*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 0.6)
skaters['Score'] = skaters['Production'] + skaters['Misc'] + skaters['Offense'] + skaters['Defense']
skaters = skaters.loc[:, ~skaters.columns.str.contains('^Unnamed')]
skaters[(skaters.Season == 20222023) & (skaters.Position == 'D')].sort_values(by='Score')[['Player', 'Date', 'GA', 'GAAA', 'xGA', 'xGAAA', 'Defense']].head(20)

,Player,Date,GA,GAAA,xGA,xGAAA,Defense
636834,Josh Morrissey,2022-10-30,1,-0.231400,3.55,-2.781400,-6.585137
636541,Bowen Byram,2022-10-29,3,-2.323133,2.75,-2.073133,-6.027004
636555,Samuel Girard,2022-10-29,3,-2.381267,2.60,-1.981267,-5.765571
659722,Drew Doughty,2023-01-27,2,-1.190867,3.30,-2.490867,-6.526871
636840,Neal Pionk,2022-10-30,1,-0.180733,3.68,-2.860733,-6.811304
659726,Mikey Anderson,2023-01-27,2,-1.252200,3.30,-2.552200,-6.615037
664519,Tyler Myers,2023-02-21,3,-2.300200,2.45,-1.750200,-5.304038
666349,Matt Grzelcyk,2023-02-28,2,-1.436733,2.28,-1.716733,-4.534304
654159,Dmitry Kulikov,2023-01-06,1,-0.282067,3.02,-2.302067,-5.438971
651333,Isaak Phillips,2022-12-27,3,-2.410600,2.04,-1.450600,-4.519738


In [ ]:
goalies = pd.read_csv(f'cleaned/master_goalies.csv')
goalies['Score'] = goalies['Expected Goals Against'] - goalies['Goals Against']
goalies.sort_values(by='Score', ascending=False)

In [ ]:
import unidecode
lst = []
for year in range(2007, 2022):
    output = tdhepscrape.get_skaters("nhl", f"{year}-{year+1}")
    results = tdhepscrape.add_player_information(output)
    results['season'] = str(year) + str(year+1)
    condensed = results[['player', 'season', 'team', 'dob']]
    lst.append(condensed)

births = pd.concat(lst)
births[['player','pos']] = births['player'].str.split('( ',expand=True)
births['pos'] = births['pos'].str.replace(')', '')
births['Team'] = births['Team'].str.replace('.', '')
births['Player'] = births['Player'].str.replace('Alex ', 'Alexander ')
births['Player'] = births['Player'].str.replace('Yevgeni ', 'Evgeni ')

decoded = [unidecode.unidecode(e) for e in births['player']]
births['player'] = decoded
births = births.rename(columns={'player':'Player', 'team':'Team', 'season':'Season'})

In [1210]:
births = pd.read_csv('birthdates.csv')

merged = skaters.merge(births[['Player', 'Season', 'dob']], on=['Player', 'Season'])
merged[["Date", "dob"]] = merged[["Date", "dob"]].apply(pd.to_datetime)
merged['Age'] = (merged["Date"] - merged["dob"]).dt.days/365


In [1212]:
from datetime import datetime
start_dates = {season: datetime.strptime((str(season)[:4] + '-10-04'), '%Y-%m-%d') for season in skaters.groupby(['Season']).count().index}

In [1373]:
import plotly.graph_objects as go
import numpy as np
from moepy import lowess
def get_test_projections(alpha):
    adjustment = 'full'
    player = 'Tage Thompson'
    df = merged[merged.Player == player]
    df = df.sort_values(by='Date')
    df['Career Game'] = range(1, len(df) + 1)
    df[["Date", "dob"]] = df[["Date", "dob"]].apply(pd.to_datetime)
    projections = [0]
    scores = np.concatenate((np.zeros(5),  df['Score'].to_list()))
    for i in range(-4, len(df)):
        curr = 0
        total = 0
        start = 1
        j = i-1
        while start > 0.05 and j >= -5:
            if j >= 0:

                weight = alpha**(df['Date'].to_list()[i] - df['Date'].to_list()[j]).days
            else:
                weight = alpha**(-1*j)
            curr += weight * scores[j+5]
            total += weight
            start = weight/total
            j -= 1

        if adjustment == 'full':
            season = df['Season'].to_list()[i]
            since_start = (df['Date'].to_list()[i] - start_dates[season]).days
            season_adjust = (-0.00109 * since_start  + 1.1)
            if df['Venue'].to_list()[i] == 'Home':
                projections.append(season_adjust*1.2*(curr/total))
            else:
                projections.append(season_adjust*0.8*(curr/total))
        elif adjustment == 'anti':
            season = df['Season'].to_list()[i]
            since_start = (df['Date'].to_list()[i] - start_dates[season]).days
            season_adjust = (-0.00109 * since_start  + 0.89)
            if df['Venue'].to_list()[i] == 'Home':
                projections.append(season_adjust*0.8*(curr/total))
            else:
                projections.append(season_adjust*1.2*(curr/total))        
        else:
            projections.append((curr/total))            

        

    df['Projection'] = projections[5:]
    df['Error'] = df['Projection'] - df['Score']
    df['AError'] = abs(df['Error'])

    return df['AError'].mean()


In [1378]:
import numpy as np

def de(fobj, bounds, mut=0.8, crossp=0.7, popsize=20, its=1000):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
        yield best, fitness[best_idx]

In [1379]:
it = list(de(lambda x: get_test_projections(x), bounds=[(0.95, 1)]))
print(it[-1])


(array([0.99968135]), array([0.96801538]))


In [1380]:
import plotly.graph_objects as go
import numpy as np
from moepy import lowess
def get_projections(player, adjustment):

    df = merged[merged.Player == player]
    df = df.sort_values(by='Date')
    df['Career Game'] = range(1, len(df) + 1)
    df['Adjusted'] = df['Score']*(50/1.55) + 50
    df[["Date", "dob"]] = df[["Date", "dob"]].apply(pd.to_datetime)
    projections = [0]
    scores = np.concatenate((np.zeros(5),  df['Score'].to_list()))
    for i in range(-4, len(df)):
        curr = 0
        total = 0
        start = 1
        j = i-1
        while start > 0.05 and j >= -5:
            if j >= 0:

                weight = 0.99968135**(df['Date'].to_list()[i] - df['Date'].to_list()[j]).days
            else:
                weight = 0.99968135**(-1*j)
            curr += weight * scores[j+5]
            total += weight
            start = weight/total
            j -= 1

        if adjustment == 'full':
            season = df['Season'].to_list()[i]
            since_start = (df['Date'].to_list()[i] - start_dates[season]).days
            season_adjust = (-0.00109 * since_start  + 1.1)
            if df['Venue'].to_list()[i] == 'Home':
                projections.append(season_adjust*1.2*(curr/total))
            else:
                projections.append(season_adjust*0.8*(curr/total))
        elif adjustment == 'anti':
            season = df['Season'].to_list()[i]
            since_start = (df['Date'].to_list()[i] - start_dates[season]).days
            season_adjust = (-0.00109 * since_start  + 0.89)
            if df['Venue'].to_list()[i] == 'Home':
                projections.append(season_adjust*0.8*(curr/total))
            else:
                projections.append(season_adjust*1.2*(curr/total))        
        else:
            projections.append((curr/total))            

        

    df['Projection'] = projections[5:]



    # Model fitting
    quant_reg_func = lowess.calc_quant_reg_betas

    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    frac = 0.35*(-0.9992**len(df)+1.6)

    quantile_model.fit(np.asarray(df['Career Game']), np.asarray(df['Projection']), frac=frac, q=0.5)

    # Model prediction
    x_pred = np.arange(0, len(df), 0.5)
    y_pred = quantile_model.predict(x_pred)

    res = {x_pred[i]: y_pred[i] for i in range(len(x_pred))}
    df['lowess'] = [res[float(i)] for i in range(0, len(df))] 
    # Model fitting
    quant_reg_func = lowess.calc_quant_reg_betas

    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df['Career Game']), np.asarray(df['Projection']), frac=frac, q=0.5, robust_iters=0)
    df_quantiles = lowess.quantile_model(np.asarray(df['Career Game']), np.asarray(df['Projection']), frac=frac, num_fits=100, robust_iters=1)
    return df, df_quantiles



In [1381]:
fig = go.Figure()
i = 0
colors = ['rgba(204, 25, 25', 'rgba(25, 114, 204', 'rgba(25, 204, 114', ]
for player in ['Matthew Tkachuk', 'Connor McDavid', 'Tage Thompson']:
    df6, df_quantiles = get_projections(player, 'full')
    fig.add_trace(go.Scatter(x=df6['Career Game'], y=df6['Projection'], mode='markers', opacity=0.15, marker=dict(color=colors[i] + ', 1)')))
    fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.8], marker=dict(color=colors[i] + ', 0.05)')))
    fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.2], fill='tonexty', fillcolor=colors[i] + ', 0.1)', marker=dict(color=colors[i] + ', 0.05)'))) # fill to trace0 y
    fig.add_trace(go.Scatter(x=df6['Career Game'], y=df6['lowess'], name=player, mode='lines', marker=dict(color=colors[i] + ', 1)'))) # fill to trace0 y

    i += 1
fig.update_layout(autosize=False, width=1600, height=500)



fig.show()

100%|██████████| 9/9 [00:03<00:00,  2.57it/s]


In [1325]:
fig = go.Figure()
i = 0
colors = ['rgba(204, 25, 25', 'rgba(25, 114, 204', 'rgba(25, 204, 114', ]
for player in ['Matthew Tkachuk', 'Connor McDavid']:
    df1, df_quantiles = get_projections(0.99, player, 'full')
    fig.add_trace(go.Scatter(x=df1['Career Game'], y=df1['Projection'], mode='markers', opacity=0.15, marker=dict(color=colors[i] + ', 1)')))
    fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.6], marker=dict(color=colors[i] + ', 0.05)')))
    fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.4], fill='tonexty', fillcolor=colors[i] + ', 0.1)', marker=dict(color=colors[i] + ', 0.05)'))) # fill to trace0 y
    fig.add_trace(go.Scatter(x=df1['Career Game'], y=df1['lowess'], name=player, mode='lines', marker=dict(color=colors[i] + ', 1)'))) # fill to trace0 y

    i += 1
fig.update_layout(autosize=False, width=1600, height=500)



fig.show()

100%|██████████| 9/9 [00:04<00:00,  2.24it/s]


In [888]:
import plotly.graph_objects as go
import numpy as np

tage = merged[merged.Player == "Patrick Kane"]
tage['Career Game'] = range(1, len(tage) + 1)
tage[["Date", "dob"]] = tage[["Date", "dob"]].apply(pd.to_datetime)
projections = [0]
scores = np.concatenate((np.zeros(5), tage['Score'].to_list()))
for i in range(-4, len(tage)):
    curr = 0
    total = 0
    start = 1
    j = i-1
    while start > 0.05 and j >= -5:
        if j >= 0:
            weight = 0.997**(tage['Date'].to_list()[i] - tage['Date'].to_list()[j]).days
        else:
            weight = 0.997**(-1*j)
        curr += weight * scores[j+5]
        total += weight
        start = weight/total
        j -= 1
    projections.append(curr/total)

tage['Projection'] = projections[5:]


fig = go.Figure(data=go.Scatter(x=tage['Career Game'], y=tage['Projection'], mode='markers'))
fig.add_trace(go.Scatter(x=tage['Career Game'], y=tage['Score'], mode='markers'))

fig.show()

In [889]:
from moepy import lowess


# Model fitting
quant_reg_func = lowess.calc_quant_reg_betas

quantile_model = lowess.Lowess(reg_func=quant_reg_func)
frac = (len(tage) + 1400)/6000


quantile_model.fit(np.asarray(tage['Career Game']), np.asarray(tage['Projection']), frac=frac, q=0.5)

# Model prediction
x_pred = np.arange(0, len(tage), 0.5)
y_pred = quantile_model.predict(x_pred)
fig = go.Figure(data=go.Scatter(x=tage['Career Game'], y=tage['Projection'], mode='markers'))

fig.add_trace(go.Scatter(x=x_pred, y=y_pred, mode='lines'))
fig.update_layout(autosize=False, width=1600, height=500)
fig.show()


In [ ]:
import numpy as np

class KalmanFilter(object):
    def __init__(self, F = None, B = None, H = None, Q = None, R = None, P = None, x0 = None):

        if(F is None or H is None):
            raise ValueError("Set proper system dynamics.")

        self.n = F.shape[1]
        self.m = H.shape[1]

        self.F = F
        self.H = H
        self.B = 0 if B is None else B
        self.Q = np.eye(self.n) if Q is None else Q
        self.R = np.eye(self.n) if R is None else R
        self.P = np.eye(self.n) if P is None else P
        self.x = np.zeros((self.n, 1)) if x0 is None else x0

    def predict(self, u = 0):
        self.x = np.dot(self.F, self.x) + np.dot(self.B, u)
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return self.x

    def update(self, z):
        y = z - np.dot(self.H, self.x)
        S = self.R + np.dot(self.H, np.dot(self.P, self.H.T))
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        self.x = self.x + np.dot(K, y)
        I = np.eye(self.n)
        self.P = np.dot(np.dot(I - np.dot(K, self.H), self.P), 
        	(I - np.dot(K, self.H)).T) + np.dot(np.dot(K, self.R), K.T)




In [890]:
res = {x_pred[i]: y_pred[i] for i in range(len(x_pred))}
tage['lowess'] = [res[float(i)] for i in range(0, len(tage))] 

In [912]:
# Model fitting
quant_reg_func = lowess.calc_quant_reg_betas

quantile_model = lowess.Lowess(reg_func=quant_reg_func)
frac = (len(tage) + 1400)/6000
quantile_model.fit(np.asarray(tage['Career Game']), np.asarray(tage['Projection']), frac=frac, q=0.5, robust_iters=0)
df_quantiles = lowess.quantile_model(np.asarray(tage['Career Game']), np.asarray(tage['Projection']), frac=frac, num_fits=100, robust_iters=1)
CI = confidence_interval(df_quantiles[0.5], 0.95)

fig = go.Figure(data=go.Scatter(x=tage['Career Game'], y=tage['Projection'], mode='markers', opacity=0.15, marker=dict(color='rgba(4, 14, 209, 1)')))
fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.9], marker=dict(color='rgba(4, 14, 209, 0.05)')))
fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.1], fill='tonexty', fillcolor='rgba(4, 14, 209, 0.1)', marker=dict(color='rgba(4, 14, 209, 0.05)'))) # fill to trace0 y
fig.add_trace(go.Scatter(x=tage['Career Game'], y=df_quantiles[0.5], mode='lines', marker=dict(color='rgba(4, 14, 209, 1)'))) # fill to trace0 y
fig.update_layout(autosize=False, width=1600, height=500)

fig.update_layout(autosize=False, width=1600, height=500)



fig.show()

100%|██████████| 9/9 [00:04<00:00,  2.04it/s]


In [ ]:
dt = 1.0/1000
F = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
H = np.array([1, 0, 0]).reshape(1, 3)
Q = np.array([[0.01, 0.01, 0.0], [0.01, 0.01, 0.0], [0.0, 0.0, 0.0]])
R = np.array([0.1]).reshape(1, 1)

x = np.linspace(-10, 10, 100)
measurements = tage['Score']

kf = KalmanFilter(F = F, H = H, Q = Q, R = R)
predictions = []
for z in measurements:
	predictions.append(np.dot(H,  kf.predict())[0])
	kf.update(z)
x = tage['Career Game'].to_list()
y = [e[0] for e in predictions]

# Model fitting
quant_reg_func = lowess.calc_quant_reg_betas

quantile_model = lowess.Lowess(reg_func=quant_reg_func)
frac = (len(tage) + 1400)/6000
quantile_model.fit(np.asarray(tage['Career Game']), np.asarray(tage['Projection']), frac=frac, q=0.5, robust_iters=0)
df_quantiles = lowess.quantile_model(np.asarray(tage['Career Game']), np.asarray(tage['Projection']), frac=frac, num_fits=100, robust_iters=1)

fig = go.Figure(data=go.Scatter(x=tage['Career Game'], y=tage['Score'], mode='markers'))
fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.6]))
fig.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.4], fill='tonexty')) # fill to trace0 y
fig.add_trace(go.Scatter(x=x, y=y)) # fill to trace0 y

fig.update_layout(autosize=False, width=1200, height=800)
tage['kalman'] = y



fig.show()

In [892]:
fig1.show()

In [895]:
fig1.add_trace(go.Scatter(x=tage['Career Game'], y=tage['Projection'], mode='markers', opacity=0.1, marker=dict(color='rgba(4, 14, 209, 1)')))
fig1.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.6], marker=dict(color='rgba(4, 14, 209, 0.15)')))
fig1.add_trace(go.Scatter(x=df_quantiles.index, y=df_quantiles[0.4], fill='tonexty', marker=dict(color='rgba(4, 14, 209, 0.15)'))) # fill to trace0 y
fig1.add_trace(go.Scatter(x=tage['Career Game'], y=tage['lowess'], mode='lines', marker=dict(color='rgba(4, 14, 209, 1)'))) # fill to trace0 y
fig1.update_layout(autosize=False, width=1600, height=500)

fig1.show()